<a href="https://colab.research.google.com/github/aissahm/image_analysis/blob/main/project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Image analysis - Project 1**

## **Introduction**

In this task we will evaluate the potential of three image operators, when used in sequence, to extract image features suitable for license plate image detection (i.e., features that must highlight the caracters of a plate). In the current directory, you will find the images of the plates and their binary masks. You will also find three random splits of the original images into training and testing sets. The task consists of the following steps, which can be applied to any of these sets.

1. Step 1: Dataset composition.
2. Step 2: Batch (z-score) normalization.
3. Step 3: Creation of a random kernel bank.
4. Step 4: Convolution with the random kernel bank. 
5. Step 5: Rectified Linear Unit activation (ReLU).

These steps are presented in the next sections.

## **Dataset composition**

You must create a subimage (patch) dataset by extracting regions of interest with size W × H pixels, center at each position (x,y) of the original images, and stride (displacement between them) of (Dx,Dy) pixels.

The plate masks (avaliable annotation) can be used in this step only to verify if your choice for W, H, Dx, and Dy is suitable to not miss any plate from the training set. That is, for the corresponding testing set, you will simply use the same parameters learned in the training set.

In this step, using the training set only, you must also define a threshold on the percentage of plate pixels that fall inside a subimage to consider it from class 1 (background) or class 2 (plate/foreground). Note that this will create a difficulty for a future plate detection since subimages with parts of a plate might be considered background. In this sense, we might need to further process those images in the future to avoid classification errors.

After creating the subimages of a same size W × H, you must convert each subimage into a multiband image by pseudo colorizing it based on a Red-Green-Blue color table (e.g., from blue to red) and then convert it into an uncorrelated color space (e.g., Y-Cb-Cr or L-a-b).
